In [18]:
from galactic_model import Nucleus
import astropy.units as u

Mbh =4.3e6 * u.Msun
mstar = 10. * u.Msun
mstarb = 1. * u.Msun

rho0 = 7.2e5 * u.Msun / u.pc**3.
rho0b = 3e6 * u.Msun / u.pc**3.

r0 = 0.1 * u.pc
gamma = 2.04
gammab = 1.46
r_s2 = 1.4 * u.Rsun
sma_s2 = 3.3e-3 * u.pc
m_s2 = 1.5 * u.Msun
e_s2 = 0.9825


nuc_bh = Nucleus(Mbh, mstar, rho0, r0, gamma)
nuc_st = Nucleus(Mbh, mstarb, rho0b, r0, gammab)

print(nuc_bh.trx(sma_s2).to("Gyr"), nuc_bh.trx(sma_s2).to("Gyr") * (1. - e_s2**2.)#, nuc_st.trx(sma_s2) / 1e9, nuc_st.trx(sma_s2) / 1e9 * (1. - e_s2**2.)
)


0.45762604572257415 Gyr 0.01587676362378754 Gyr


In [23]:
import sys
sys.path.append("/home/aleksey/code/python")
import pickle 
import cgs_const as cgs
from jrx_mc.diff_ang_opt_rearrange import get_diffusionNR, get_diffusionRR

diff_file = "diff_data_linear_m10.p"
with open(diff_file, "rb") as ff:
    diff_data = pickle.load(ff)


j_s2 = (1. - ecc_s2**2.)**.5
DaNR, DaaNR, DjNR, DjjNR, DajNR = get_diffusionNR(diff_data, 1.5, sma_s2.to("mpc").value, np.log(j_s2), 4.3e6)
DjRR, DjjRR = get_diffusionRR(diff_data, 1.5, sma_s2.to("mpc").value, np.log(j_s2))


print((sma_s2.to('pc').value * cgs.pc)**2./ DaaNR / 1e6 / cgs.year)
print(j_s2**2. / (DjjNR + DjjRR) / 1e6 / cgs.year)

FileNotFoundError: [Errno 2] No such file or directory: 'diff_data_linear_m10.p'

In [ ]:
# import numpy as np

# np.log(4.3e6/ 10.)

# (nuc_bh.coll_rate(sma_s2, r_s2, m_s2)), 

np.float64(12.971540487669746)

In [68]:
import rebound
import numpy as np

sim = rebound.Simulation()
sim.add(m=Mbh.value)
sim.add(m=m_s2.value, a=sma_s2.value, e=e_s2)

N1 = [100, 1000, 10000]

def sample_orbit(r, e, N):
    absc = np.linspace(0, 2. * np.pi, N)
    xyz = np.vstack((r * (np.cos(absc) - e), r * (1. - e * e) ** .5 * np.sin(absc), np.zeros(len(absc))))
    weights = (1. - e * np.cos(absc))

    return xyz.T, weights

res = []
for NN in N1:
    # pos = sim.particles[1].sample_orbit(Npts=NN, samplingAngle="M")

    pos, weights = sample_orbit(sma_s2.value, e_s2, NN)
    pos = np.array(pos)
    rs = np.sum(pos * pos, axis=1)**.5
    mean_rate = np.average([nuc_bh.coll_rate(rr * u.pc, 1.6 * u.Rsun, 1.5 * u.Msun).value for rr in rs], weights=weights)
    res.append(mean_rate)

In [69]:
res

[np.float64(1.6360043316125533e-08),
 np.float64(1.4322848153600448e-08),
 np.float64(1.4121110484090475e-08)]

In [70]:
1.4e-8 / 1.07e-9

13.084112149532709